In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from transformers import BertTokenizer, BertModel
from datasets import load_dataset

from tqdm import tqdm # Thư viện hữu ích để xem thanh tiến trình
import warnings     # Để tắt các cảnh báo không cần thiết
import os

# Cài đặt
warnings.filterwarnings('ignore') # Bỏ qua các cảnh báo
os.environ["TOKENIZERS_PARALLELISM"] = "false" # Tắt cảnh báo của tokenizer

c:\Users\Dell5530\Sentiment-Analysis\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# config.py
BERT_MODEL_NAME = 'bert-base-uncased'
DATASET_NAME = 'SetFit/sst5'

# Quyết định từ file 1.0
MAX_LENGTH = 64 

# Cấu hình thử nghiệm
BATCH_SIZE = 8 # Chỉ cần 8 mẫu để thử
LEARNING_RATE = 2e-5

# Cấu hình mô hình
LSTM_HIDDEN_SIZE = 256 # Kích thước lớp ẩn LSTM
LSTM_LAYERS = 2      # Số lớp BiLSTM (stacked)
DROPOUT_RATE = 0.3
NUM_CLASSES = 5      # 5 lớp sentiment

# Thiết bị (rất quan trọng)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {DEVICE}")

Using device: cpu


In [3]:
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_NAME)

# Test thử tokenizer
test_text = "This movie is not bad, it's actually great!"
encoding = tokenizer.encode_plus(
    test_text,
    add_special_tokens=True,    # Thêm [CLS] và [SEP]
    max_length=MAX_LENGTH,      # Pad hoặc Truncate
    padding='max_length',       # Pad tới max_length
    truncation=True,            # Cắt nếu dài hơn max_length
    return_tensors='pt'         # Trả về PyTorch tensor
)

print("--- Test Tokenizer ---")
print("Input IDs shape:", encoding['input_ids'].shape)
print("Attention Mask shape:", encoding['attention_mask'].shape)
print("-" * 30)

--- Test Tokenizer ---
Input IDs shape: torch.Size([1, 64])
Attention Mask shape: torch.Size([1, 64])
------------------------------


In [4]:
class SST5PrototypeDataset(Dataset):
    def __init__(self, data, tokenizer, max_length):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        # Lấy mẫu dữ liệu
        sample = self.data[idx]
        text = sample['text']
        label = sample['label']
        
        # Tokenize văn bản
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        # .squeeze() để loại bỏ chiều batch (vì tokenizer trả về [1, max_length])
        # .to(torch.long) là cần thiết cho label
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'label': torch.tensor(label, dtype=torch.long)
        }

In [5]:
class BertLSTMClassifier(nn.Module):
    def __init__(self):
        super(BertLSTMClassifier, self).__init__()
        
        # --- 1. Lớp BERT (Feature Extractor) ---
        self.bert = BertModel.from_pretrained(BERT_MODEL_NAME)
        # **ĐÓNG BĂNG BERT**
        # không train lại BERT, chỉ dùng nó để trích xuất đặc trưng
        for param in self.bert.parameters():
            param.requires_grad = False
            
        bert_output_size = self.bert.config.hidden_size # (Đây là 768)
        
        # --- 2. Lớp Stacked BiLSTM (Encoder) ---
        self.lstm = nn.LSTM(
            input_size=bert_output_size,         # Đầu vào là 768
            hidden_size=LSTM_HIDDEN_SIZE,  # 256
            num_layers=LSTM_LAYERS,        # 2
            bidirectional=True,                # BiLSTM
            batch_first=True,                  # Rất quan trọng! [Batch, Seq, Feature]
            dropout=DROPOUT_RATE if LSTM_LAYERS > 1 else 0
        )
        
        lstm_output_size = LSTM_HIDDEN_SIZE * 2 # (256 * 2 = 512, vì là BiLSTM)
        
        # --- 3. Lớp Attention ---
        # Chúng ta sẽ học một "trọng số" cho mỗi hidden state của LSTM
        # (cơ chế Attention đơn giản và phổ biến)
        # W*h + b
        self.attention_weights = nn.Linear(lstm_output_size, 1)
        
        # --- 4. Lớp Phân loại (Classifier) ---
        self.dropout = nn.Dropout(DROPOUT_RATE)
        self.classifier = nn.Linear(
            lstm_output_size, # 512
            NUM_CLASSES       # 5
        )

    def forward(self, input_ids, attention_mask):
        # 1. BERT: [B, S] -> [B, S, 768] (B=Batch Size, S=Seq Length)
        # last_hidden_state chính là features
        bert_output = self.bert(
            input_ids=input_ids, 
            attention_mask=attention_mask
        )
        bert_features = bert_output.last_hidden_state
        
        # 2. BiLSTM: [B, S, 768] -> [B, S, 512]
        # lstm_output chứa hidden state của *tất cả* các time step
        lstm_output, (h_n, c_n) = self.lstm(bert_features)
        
        # 3. Attention: [B, S, 512] -> [B, 512]
        # Tính "điểm số" (logits) cho mỗi từ
        # (B, S, 512) -> (B, S, 1)
        attn_logits = self.attention_weights(lstm_output)
        
        # Chúng ta không muốn Attention chú ý vào các token [PAD]
        # Tạo một mask ngược lại từ attention_mask của BERT
        # (B, S) -> (B, S, 1)
        attn_mask = attention_mask.unsqueeze(2)
        # Fill các vị trí [PAD] (mask=0) bằng -infinity để softmax = 0
        attn_logits = attn_logits.masked_fill(attn_mask == 0, -1e9)
        
        # Áp dụng softmax để ra trọng số
        # (B, S, 1)
        attn_scores = F.softmax(attn_logits, dim=1) 
        
        # Tính "vector ngữ cảnh" (weighted sum)
        # (B, S, 512) * (B, S, 1) -> (B, S, 512)
        # .sum(dim=1) -> (B, 512)
        context_vector = torch.sum(lstm_output * attn_scores, dim=1)
        
        # 4. Classifier: [B, 512] -> [B, 5]
        context_vector = self.dropout(context_vector)
        logits = self.classifier(context_vector)
        
        return logits

In [6]:
print("--- BẮT ĐẦU KIỂM THỬ PIPELINE ---")

try:
    # 1. Tải một phần nhỏ của dữ liệu
    print(f"[1/8] Đang tải {BATCH_SIZE * 2} mẫu từ '{DATASET_NAME}'...")
    raw_datasets = load_dataset(DATASET_NAME)
    small_train_data = raw_datasets['train'].select(range(BATCH_SIZE * 2))
    print("     Tải xong!")

    # 2. Tạo Dataset
    print("[2/8] Đang tạo Prototype Dataset...")
    train_dataset = SST5PrototypeDataset(
        data=small_train_data,
        tokenizer=tokenizer,
        max_length=MAX_LENGTH
    )

    # 3. Tạo DataLoader
    print("[3/8] Đang tạo DataLoader...")
    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True
    )

    # 4. Lấy MỘT BATCH dữ liệu
    print("[4/8] Đang lấy 1 batch dữ liệu...")
    batch = next(iter(train_loader))
    
    # Chuyển batch lên DEVICE
    input_ids = batch['input_ids'].to(DEVICE)
    attention_mask = batch['attention_mask'].to(DEVICE)
    labels = batch['label'].to(DEVICE)
    
    print(f"     Input IDs shape: {input_ids.shape}")
    print(f"     Attention Mask shape: {attention_mask.shape}")
    print(f"     Labels shape: {labels.shape}")
    
    assert input_ids.shape == (BATCH_SIZE, MAX_LENGTH)

    # 5. Khởi tạo Mô hình (dùng class ở Cell 5)
    print("[5/8] Đang khởi tạo mô hình BertLSTMClassifier...")
    model = BertLSTMClassifier().to(DEVICE)
    # print(model) # Bỏ comment nếu xem kiến trúc

    # 6. Khởi tạo Optimizer và Loss (dùng nn và torch.optim từ Cell 1)
    print("[6/8] Đang khởi tạo Optimizer và Loss Function...")
    optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
    criterion = nn.CrossEntropyLoss()

    # 7. CHẠY FORWARD PASS
    print("[7/8] Đang thực hiện Forward Pass (model(inputs))...")
    optimizer.zero_grad()
    logits = model(input_ids, attention_mask)
    
    print(f"     Output Logits shape: {logits.shape}")
    assert logits.shape == (BATCH_SIZE, NUM_CLASSES)
    print("     Forward Pass THÀNH CÔNG!")

    # 8. TÍNH LOSS VÀ BACKWARD PASS
    print("[8/8] Đang thực hiện Backward Pass (loss.backward())...")
    loss = criterion(logits, labels)
    loss.backward()
    optimizer.step()
    
    print(f"     Tính được Loss: {loss.item():.4f}")
    print("     Backward Pass THÀNH CÔNG!")
    
    print("\n--- KIỂM THỬ HOÀN TẤT --- 🎉")
    print("Toàn bộ pipeline hoạt động tốt. Kích thước tensor chính xác.")
    print("Sẵn sàng để chuyển code này sang các file .py trong 'src/'!")

except Exception as e:
    print(f"\n--- !!! GẶP LỖI !!! ---")
    print(f"Lỗi: {e}")
    import traceback
    traceback.print_exc()

--- BẮT ĐẦU KIỂM THỬ PIPELINE ---
[1/8] Đang tải 16 mẫu từ 'SetFit/sst5'...


Repo card metadata block was not found. Setting CardData to empty.


     Tải xong!
[2/8] Đang tạo Prototype Dataset...
[3/8] Đang tạo DataLoader...
[4/8] Đang lấy 1 batch dữ liệu...
     Input IDs shape: torch.Size([8, 64])
     Attention Mask shape: torch.Size([8, 64])
     Labels shape: torch.Size([8])
[5/8] Đang khởi tạo mô hình BertLSTMClassifier...
[6/8] Đang khởi tạo Optimizer và Loss Function...
[7/8] Đang thực hiện Forward Pass (model(inputs))...
     Output Logits shape: torch.Size([8, 5])
     Forward Pass THÀNH CÔNG!
[8/8] Đang thực hiện Backward Pass (loss.backward())...
     Tính được Loss: 1.6324
     Backward Pass THÀNH CÔNG!

--- KIỂM THỬ HOÀN TẤT --- 🎉
Toàn bộ pipeline hoạt động tốt. Kích thước tensor chính xác.
Sẵn sàng để chuyển code này sang các file .py trong 'src/'!
